# Corrects mis-attributed RGI sub-region attributes

Some glaciers (140) have been attributed to the wrong RGI sub-region number. The majority (116) are nominal glacier at high lats, but some other are simply close to the border and were obviously mis-attributed.

This script corrects the shapefiles' attributes and writes the files in a new directory.

In [5]:
out_dir = 'RGI62'
in_dir = 'RGI62_update_logs'
rgi_version = '61'
replace_str = lambda x : x.replace('rgi61', 'rgi62')

In [2]:
import os
import geopandas as gpd
from oggm import utils

In [3]:
in_dir = os.path.abspath(in_dir)
out_dir = os.path.abspath(out_dir)

In [4]:
# The wrong region
reg_cor = gpd.read_file(os.path.join(in_dir, 'entities_misplaced', 'subreg_misplaced_after_cor.shp'))
reg_cor = reg_cor[['RGIId', 'O1Region', 'O2Region', 'NEW_REG', 'NEW_SREG']].copy()
reg_cor['NEW_SREG'] = [str(int(s.split('-')[1])) for s in reg_cor['NEW_SREG']]
reg_cor.head()

,RGIId,O1Region,O2Region,NEW_REG,NEW_SREG
0,RGI60-01.12756,1,5,None,6
1,RGI60-01.15201,1,2,None,5
2,RGI60-01.15481,1,2,None,5
3,RGI60-01.15482,1,2,None,5
4,RGI60-01.15485,1,2,None,5


In [6]:
# Loop over RGI files
out = []
for reg in range(1, 20):
    # Read in
    f = utils.get_rgi_region_file('{:02d}'.format(reg), version=rgi_version)
    sh = gpd.read_file(f)
    
    # Replace
    _reg_cor = reg_cor.loc[reg_cor.O1Region == sh.iloc[0].O1Region]
    ides = sh.RGIId.isin(_reg_cor.RGIId)
    if ides.sum() > 0:
        sh.loc[ides, 'O2Region'] = _reg_cor.NEW_SREG.values
        
    # Write out
    of = os.path.join(out_dir, os.path.basename(f).replace('.shp', ''))
    utils.mkdir(replace_str(of))
    of = os.path.join(of, os.path.basename(f))
    sh.to_file(replace_str(of))